In [1]:
#import libaries
import pandas as pd
import numpy as np
import requests
import json
import os
import sqlalchemy
import time
from tqdm.notebook import tqdm
from io import StringIO
from google.cloud import storage

In [2]:
#extract data function
def extract_data(database_name, table):
    engine = sqlalchemy.create_engine(f'mysql+pymysql://root@localhost/{database_name}')
    query = f'SELECT * FROM {table};'
    chunk_size = 100000
    counter = 0
    all_chunks = []
    #read by chunk
    for chunk in pd.read_sql_query(query, engine, chunksize=chunk_size):
        start_time = time.time()
        print(f'Processing Chunk {counter}...')
        all_chunks.append(chunk)
        counter += 1
        end_time = time.time()
        total_time = end_time - start_time
        print(f'Chunk {counter} processed in {total_time:.2f} seconds')
    if all_chunks:
        df = pd.concat(all_chunks, ignore_index=True)
    else:
        df = pd.DataFrame()
    return df

In [3]:
#function to store data to the google cloud
def store_to_google_bucket(df, df_name):
    try:
        service_account_key_path = 'different-state-hospital-price-fd662d2f48c2.json'
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_key_path
        # Initialize the GCP storage client
        client = storage.Client()
        json_data = df.to_json(orient='records', lines=True)
        bucket_name = 'different-state-hospital-prices'
        destination_blob_name = f'{df_name}.json'
        # Get the bucket object
        bucket = client.get_bucket(bucket_name)
        # Create a new blob and upload the JSON data
        blob = bucket.blob(destination_blob_name)
        blob.upload_from_string(json_data, content_type='application/json')
        print(f'Data successfully uploaded to {destination_blob_name}')
    except Exception as e:
        print(f'An error occurred: {e}')

In [4]:
cpt_hcpcs_df = extract_data('hospital-price-transparency', 'cpt_hcpcs')
print(cpt_hcpcs_df.shape)
cpt_hcpcs_df.head()

Processing Chunk 0...
Chunk 1 processed in 0.00 seconds
Processing Chunk 1...
Chunk 2 processed in 0.00 seconds
Processing Chunk 2...
Chunk 3 processed in 0.00 seconds
Processing Chunk 3...
Chunk 4 processed in 0.00 seconds
Processing Chunk 4...
Chunk 5 processed in 0.00 seconds
Processing Chunk 5...
Chunk 6 processed in 0.00 seconds
Processing Chunk 6...
Chunk 7 processed in 0.00 seconds
Processing Chunk 7...
Chunk 8 processed in 0.00 seconds
Processing Chunk 8...
Chunk 9 processed in 0.00 seconds
Processing Chunk 9...
Chunk 10 processed in 0.00 seconds
Processing Chunk 10...
Chunk 11 processed in 0.00 seconds
Processing Chunk 11...
Chunk 12 processed in 0.00 seconds
Processing Chunk 12...
Chunk 13 processed in 0.00 seconds
Processing Chunk 13...
Chunk 14 processed in 0.00 seconds
Processing Chunk 14...
Chunk 15 processed in 0.00 seconds
Processing Chunk 15...
Chunk 16 processed in 0.00 seconds
Processing Chunk 16...
Chunk 17 processed in 0.00 seconds
Processing Chunk 17...
Chunk 18 p

,code,short_description,long_description
0,00000A,DVC REVASC 6X20MM 200CM,None
1,00001U,RBC DNA HEA 35 AG PLA,None
2,"00001U,1",RBC DNA HEA 35 AG PLA,None
3,00013,PT INDIVIDUAL GYM,None
4,0001A,HC ADM PFIZER SARSCOV2 30MCG/0.3ML 1ST,None


In [5]:
store_to_google_bucket(cpt_hcpcs_df, 'cpt_hcpcs')

Data successfully uploaded to cpt_hcpcs.json


In [6]:
hospitals_df = extract_data('hospital-price-transparency', 'hospitals')
print(hospitals_df.shape)
hospitals_df.head()

Processing Chunk 0...
Chunk 1 processed in 0.00 seconds
(1400, 8)


,npi_number,name,url,street_address,city,state,zip_code,publish_date
0,1003139775.0,HCA Virginia,https://hcavirginia.com/about/legal/pricing-tr...,901 E. Cary St Suite 210,Richmond,VA,None,2021-01-01
1,1003260480,Brookwood Baptist Medical Center,https://www.brookwoodbaptisthealth.com/docs/gl...,2010 Brookwood Medical Center Dr.,Birmingham,AL,35209,None
2,1003281452,Henderson Hospital,https://uhsfilecdn.eskycity.net/ac/henderson-h...,1050 West Galleria Drive,Henderson,NV,89011,2021-01-01
3,1003362997,CHI Health St. Elizabeth,https://www.chihealth.com/content/dam/chi-heal...,555 S. 70Th St.,Lincoln,NE,68510,2021-01-01
4,1003389206,Merrill pioneer hospital,https://www.avera.org/app/files/public/79147/m...,"1100 S 10th Ave, Ste 100",Rock Rapids,IA,51246-2020,None


In [7]:
store_to_google_bucket(hospitals_df, 'hospitals')

Data successfully uploaded to hospitals.json


In [ ]:
prices_df = extract_data('hospital-price-transparency', 'prices')
print(prices_df.shape)
prices_df.head()

In [ ]:
store_to_google_bucket(prices_df, 'prices')

In [2]:
loop_numbers = list(range(1, 138))  # Loop numbers from 1 to 137
time_taken = [
    0.08, 0.12, 0.17, 0.18, 0.21, 0.21, 0.24, 0.25, 0.27, 0.31, 0.43, 0.43, 0.42, 0.43, 0.49, 0.62, 0.59, 0.61, 0.57, 0.65,
    0.68, 0.61, 0.69, 0.66, 0.70, 0.72, 0.73, 0.76, 0.75, 0.80, 0.81, 0.90, 0.88, 0.89, 0.87, 0.91, 0.92, 0.96, 0.99, 1.02, 1.07,
    1.00, 1.02, 1.03, 1.13, 1.08, 1.13, 1.14, 1.13, 1.18, 1.17, 1.24, 1.22, 1.30, 1.25, 1.31, 1.28, 1.35, 1.40, 1.36, 1.43, 1.44,
    1.50, 1.48, 1.54, 1.49, 1.53, 1.55, 1.58, 1.54, 1.58, 1.64, 1.64, 1.65, 1.66, 1.69, 1.73, 1.76, 1.83, 1.85, 1.79, 1.82, 1.86,
    1.87, 1.92, 1.90, 1.99, 1.98, 2.01, 2.01, 2.06, 2.09, 2.07, 2.11, 2.16, 2.13, 2.19, 2.17, 2.24, 2.25, 2.25, 2.26, 2.43, 2.73,
    2.34, 2.37, 2.40, 2.47, 2.43, 2.45, 2.52, 2.55, 2.54, 2.60, 2.57, 2.64, 2.66, 2.75, 2.72, 2.75, 2.76, 2.78, 2.79, 2.81, 2.86,
    2.81, 2.92, 2.85, 2.96, 2.88, 3.24, 2.98, 3.01, 3.22, 3.04, 3.12, 3.27
]

from sklearn.linear_model import LinearRegression

# Reshape the data for sklearn
import numpy as np
X = np.array(loop_numbers).reshape(-1, 1)
y = np.array(time_taken)

# Fit linear regression model
model = LinearRegression()
model.fit(X, y)

predicted_time_723_loops = model.predict([[723]])[0]
print("Predicted time for 723 loops:", predicted_time_723_loops, "seconds")

predicted_time_one_loop = model.predict([[1]])[0]
total_time_723_loops = predicted_time_one_loop * 723
print("Total time for 723 loops:", total_time_723_loops, "seconds")

Predicted time for 723 loops: 15.778897735546138 seconds
Total time for 723 loops: 100.11634084417669 seconds
